In [1]:
import os
import zipfile
import shutil
import subprocess





In [2]:
#!/usr/bin/env python3
"""
This script provides utilities for:

1. Converting fMRI volumes into AAL3-based parcellations and saving time-series data (.dat files).
2. Visualizing the AAL3 atlas.
3. Generating example subsequences from fMRI data for modeling or analysis.

Author: [Your Name]
Date: [Optional: YYYY-MM-DD]

Dependencies:
- numpy
- nibabel
- nilearn
- matplotlib
- ipdb (optional, for debugging)
- pandas
- python >= 3.6
"""

import os
import argparse
import glob
import pickle

import numpy as np
import nibabel as nib
from nilearn import plotting
import matplotlib.pyplot as plt

from ipdb import set_trace




In [3]:
# def convert_fMRIvols_to_AAL3(data_path, output_path):
#     """
#     Convert 4D fMRI volumes into AAL3 parcellations.

#     This function:
#       1) Scans a given directory (data_path) for .nii.gz files.
#       2) For each matching file, it loads an AAL3 atlas (hardcoded path) and the fMRI data.
#       3) Reshapes the 4D fMRI volume into [n_voxels, n_timepoints], then extracts
#          average time-series data for each AAL3 parcel (1 to 170).
#       4) Saves the resulting time-series as a .dat file in 'output_path'.

#     Args:
#         data_path (str): Directory containing preprocessed fMRI volumes (.nii.gz).
#         output_path (str): Directory where the parcellated time-series (.dat) files are saved.

#     Notes:
#         - The path for the AAL3 atlas is currently hardcoded as:
#             '/orange/ruogu.fang/zeyun.zhao/FSL/bb_FSL/data/standard/AAL/AAL3.nii.gz'
#         - The function is set to skip files unless they match a specific substring
#           ("1000023_20227_2_0_fMRI_in_MNI_space.nii.gz") in this example code; adapt as needed.
#         - Each .dat file is named after the original fMRI filename with '.nii.gz' removed.

#     Returns:
#         None
#     """
#     paths = os.listdir(data_path)
#     print("fMRI data path specified:", data_path)
#     print("Number of fMRI files found:", len(paths))

#     # Hardcoded AAL3 atlas path
#     aal_path = '/orange/ruogu.fang/zeyun.zhao/FSL/bb_FSL/data/standard/AAL/AAL3.nii.gz'
#     print("Atlas file:", aal_path)

#     # Load the atlas
#     try:
#         label_img = nib.load(aal_path)
#         label_data = label_img.get_fdata()
#         label_data = label_data.flatten()  # Flatten into 1D
#         print("Atlas successfully loaded.")
#     except Exception as e:
#         print(f'Error loading AAL3 atlas at {aal_path}: {str(e)}')
#         return

#     # Loop over all files in the directory
#     for f in paths:
#         file_path = os.path.join(data_path, f)

#         # Example filter: proceed only if filename contains a specific substring
#         if "1000023_20227_2_0_fMRI_in_MNI_space.nii.gz" not in f:
#             continue

#         # Process only .nii.gz files
#         if ".nii.gz" in f:
#             print(f'Loading 4D image from {file_path}')
#             try:
#                 dts_img = nib.load(file_path)
#                 dts_data = dts_img.get_fdata()
#                 print("Loaded fMRI data.")
#             except Exception as e:
#                 print(f'Error loading 4D fMRI file "{f}": {str(e)}')
#                 continue

#             try:
#                 print(f"Extracting AAL3 parcels for {f}...")
#                 # Reshape from (X, Y, Z, T) to (T, X*Y*Z)
#                 flattened = dts_data.reshape((-1, dts_data.shape[-1])).T
#                 n_timepoints = flattened.shape[0]

#                 # AAL3 has 170 parcels (labeled 1 through 170)
#                 n_parcels = 170
#                 pmTS = np.zeros((n_timepoints, n_parcels))

#                 # Compute mean signal for each parcel i
#                 # The label_data is also flattened (same shape in spatial dims).
#                 for i in range(1, n_parcels + 1):
#                     parcel_mask = (label_data == i)
#                     y = flattened[:, parcel_mask]
#                     pmTS[:, i - 1] = np.nanmean(y, axis=1)

#                 # Replace NaNs with 0
#                 pmTS[np.isnan(pmTS)] = 0

#                 # Save time series as .dat
#                 save_name = f.split('.nii.gz')[0]
#                 out_file = os.path.join(output_path, f'{save_name}.dat')
#                 print(f"Saving {out_file} with shape {pmTS.shape} (timepoints x parcels).")
#                 np.savetxt(out_file, pmTS, delimiter='\t')

#                 set_trace()  # Debug if needed
#             except Exception as e:
#                 print(f"Error extracting or saving parcels for {f}: {str(e)}")
#         else:
#             print(f"Skipping non-NIfTI file: {f}")


# def show_AAL3(aal_template_path, save_dir):
#     """
#     Visualize each region in the AAL3 atlas by generating separate PNGs.

#     This function:
#       - Loads the AAL3 template.
#       - Iterates over all possible region indices (0 to 169 in this script).
#       - For each region, creates a binary mask image, then uses nilearn.plotting.plot_roi
#         to produce an orthographic display, saved as a PNG.

#     Args:
#         aal_template_path (str): Path to the AAL3 atlas NIfTI file (e.g., AAL3.nii.gz).
#         save_dir (str): Directory where the visualization PNG files are saved.

#     Returns:
#         None
#     """
#     output_image_path = os.path.join(save_dir, "AAL.jpg")

#     try:
#         label_img = nib.load(aal_template_path)
#         label_data = label_img.get_fdata().reshape(91, 109, 91, 1)
#     except Exception as e:
#         print(f"Error loading AAL3 atlas at {aal_template_path}: {str(e)}")
#         return

#     set_trace()  # Debug if needed

#     # AAL3 has 170 regions by default; adjust if needed
#     for roi_index in range(170):
#         # Create a binary mask for the current ROI
#         roi_mask_data = (label_data == roi_index).astype(np.int16)
#         roi_mask_img = nib.Nifti1Image(roi_mask_data, affine=label_img.affine)

#         # Show ROI in an orthographic view
#         display = plotting.plot_roi(
#             roi_mask_img,
#             title=f"ROI Index {roi_index}",
#             display_mode='ortho',
#             colorbar=True
#         )
#         # Save figure
#         output_path = os.path.join(save_dir, f"{roi_index}.png")
#         display.savefig(output_path)
#         display.close()


# def generate_subsequences(fmri_data, subsequence_length=200, segment_length=20, num_segments=10):
#     """
#     Sample random subsequences and segment them for each region of the fMRI data.

#     Given an fMRI dataset shaped (timepoints, regions):
#       1) Randomly sample a subsequence of length = subsequence_length (default 200) from the time dimension.
#       2) Split that subsequence into multiple (num_segments) segments, each of length segment_length.

#     Args:
#         fmri_data (np.ndarray): fMRI data shaped (T, R), where T=number of timepoints, R=number of regions.
#         subsequence_length (int): Length of the randomly sampled subsequence (default=200).
#         segment_length (int): Size of each segment (default=20).
#         num_segments (int): Number of segments per subsequence (default=10).

#     Returns:
#         list of np.ndarray:
#             A list containing one set of segments for each region. Each set is
#             a list of `num_segments` arrays, each of shape (segment_length,).

#     Raises:
#         AssertionError: If the split does not produce the expected number of segments.
#     """
#     num_timesteps, num_regions = fmri_data.shape
#     subsequences = []

#     set_trace()  # Debug if needed

#     for i in range(num_regions):
#         # Random start index for the subsequence
#         start_idx = np.random.randint(0, num_timesteps - subsequence_length)
#         # Extract the subsequence for region i
#         subsequence = fmri_data[start_idx:start_idx + subsequence_length, i]
#         # Split the subsequence into smaller segments
#         segments = [subsequence[j:j + segment_length] for j in range(0, subsequence_length, segment_length)]
#         # Check we have exactly num_segments segments
#         assert len(segments) == num_segments, f"Expected {num_segments} segments, got {len(segments)}"
#         subsequences.append(segments)

#     return subsequences


# def main():
#     """
#     Main execution flow for testing and demonstration.

#     1) Defines paths for the AAL3 template, an fMRI data directory, and an output directory.
#     2) Optionally calls the show_AAL3() function to visualize the AAL3 atlas.
#     3) Optionally calls the convert_fMRIvols_to_AAL3() function to parcellate .nii.gz data into .dat files.
#     4) Demonstrates how to generate random subsequences from artificially-created fMRI data (490 timepoints, 90 regions).
#     """
#     aal_template_path = '/orange/ruogu.fang/zeyun.zhao/FSL/bb_FSL/data/standard/AAL/AAL3.nii.gz'  
#     output_path = "/orange/ruogu.fang/zeyun.zhao/DATA/UKB_sub/rsfMRI_processed_nii/imgs"
#     fmri_data_path = '/orange/ruogu.fang/zeyun.zhao/DATA/UKB_sub/rsfMRI_processed_nii/Affined'

#     # Uncomment if you want to visualize AAL3 regions
#     # show_AAL3(aal_template_path, output_path)

#     # Uncomment if you want to convert volumes in fmri_data_path
#     # convert_fMRIvols_to_AAL3(fmri_data_path, output_path)

#     # Example usage of generate_subsequences:
#     print("Generating random example subsequences from mock data...")
#     fmri_data = np.random.rand(490, 90)  # (timepoints=490, regions=90)
#     subsequences = generate_subsequences(fmri_data)
#     set_trace()




In [4]:
import os
import numpy as np
import nibabel as nib
from typing import List



In [8]:

# def list_nifti_files(
#     data_path: str,
#     filename_substring: str = None,
#     extension: str = ".nii.gz"
# ) -> List[str]:
#     """
#     Return a list of file paths in `data_path` that match the given extension
#     and optionally contain a specified substring in the filename.
#     """
#     files = []
#     for fname in os.listdir(data_path):
#         if fname.endswith(extension):
#             if filename_substring is None or filename_substring in fname:
#                 files.append(os.path.join(data_path, fname))
#     return files

def list_nifti_files(
    data_path: str,
    filename_substring: str = None,
    extension: str = ".nii.gz"
) -> List[str]:
    """
    Recursively search `data_path` for files matching a specific extension
    (default: .nii.gz) and optionally filter by a substring in the filename.

    Args:
        data_path (str): The root directory to search.
        filename_substring (str, optional): Only return files that contain this substring.
        extension (str, optional): The file extension to match (default: '.nii.gz').

    Returns:
        List[str]: A list of full file paths that match the criteria.
    """
    # Construct a pattern to match all files with the given extension in any subdirectory
    pattern = os.path.join(data_path, '**', f'*{extension}')
    
    # Use glob with recursive=True to find all matching files
    matched_files = glob.glob(pattern, recursive=True)

    # If a substring is specified, filter further
    if filename_substring is not None:
        matched_files = [
            f for f in matched_files
            if filename_substring in os.path.basename(f)
        ]

    return matched_files

def load_atlas(atlas_path: str) -> np.ndarray:
    """
    Load a NIfTI atlas from `atlas_path` and return it as a flattened NumPy array.
    """
    try:
        atlas_img = nib.load(atlas_path)
        atlas_data = atlas_img.get_fdata()
        return atlas_data.flatten()
    except Exception as e:
        raise IOError(f"Failed to load atlas at {atlas_path}: {str(e)}")

def extract_parcel_timeseries(
    fmri_path: str,
    label_data: np.ndarray,
    n_parcels: int = 170
) -> np.ndarray:
    """
    Load a 4D fMRI volume from `fmri_path`, reshape it, and compute mean time series
    for each of the `n_parcels` in `label_data`. Assumes labels 1..n_parcels.
    Returns a 2D array of shape (timepoints, parcels).
    """
    fmri_img = nib.load(fmri_path)
    fmri_data = fmri_img.get_fdata()
    
    # Flatten spatial dimensions, transpose so shape is (timepoints, voxels)
    flattened = fmri_data.reshape((-1, fmri_data.shape[-1])).T
    n_timepoints = flattened.shape[0]

    # Initialize output array
    pmTS = np.zeros((n_timepoints, n_parcels))

    for i in range(1, n_parcels + 1):
        parcel_mask = (label_data == i)
        y = flattened[:, parcel_mask]  # shape: (timepoints, #voxels_in_parcel)
        pmTS[:, i - 1] = np.nanmean(y, axis=1)

    # Replace NaNs with 0
    pmTS[np.isnan(pmTS)] = 0
    return pmTS

def save_timeseries(pmTS: np.ndarray, out_file: str):
    """
    Save time series (2D NumPy array) to a .dat file using tab delimiters.
    """
    np.savetxt(out_file, pmTS, delimiter='\t')

def convert_fMRIvols_to_parcels(
    data_path: str,
    atlas_path: str,
    filename_substring: str = None,
    n_parcels: int = 170
) -> None:
    """
    Convert 4D fMRI volumes into parcel-based time series using a provided atlas,
    and store the output .dat files in the SAME folder as the input .nii.gz files.

    Steps:
      1) List .nii.gz files in `data_path`. Optionally filter by `filename_substring`.
      2) Load `atlas_path` as a flattened array of integer labels.
      3) For each fMRI file, load and reshape the 4D volume to (voxels x time).
      4) Compute average time series for each parcel label (1..n_parcels).
      5) Save the resulting matrix as a .dat file in the same folder as the input fMRI file.

    Args:
        data_path (str): Directory of 4D fMRI .nii.gz files.
        atlas_path (str): Path to the NIfTI atlas (e.g., AAL3).
        filename_substring (str, optional): Only process files containing this substring.
        n_parcels (int, optional): Number of parcels expected in the atlas.

    Returns:
        None
    """
    fmri_files = list_nifti_files(data_path, filename_substring)
    print(f"Found {len(fmri_files)} files in '{data_path}' matching criteria.")

    # Load the atlas
    try:
        label_data = load_atlas(atlas_path)
        print(f"Atlas loaded successfully from: {atlas_path}")
    except IOError as e:
        print(str(e))
        return

    for fmri_path in fmri_files:
        filename = os.path.basename(fmri_path)
        print(f"Processing file: {filename}")

        try:
            pmTS = extract_parcel_timeseries(fmri_path, label_data, n_parcels=n_parcels)
        except Exception as e:
            print(f"Error extracting time series for {filename}: {str(e)}")
            continue

        # Build output path in the SAME folder
        out_dir = os.path.dirname(fmri_path)
        base_name = filename.replace(".nii.gz", "")
        out_file = os.path.join(out_dir, f"{base_name}.dat")

        try:
            save_timeseries(pmTS, out_file)
            print(f"Saved parcellated time series to: {out_file}")
        except Exception as e:
            print(f"Error saving time series for {filename}: {str(e)}")


# Example usage (comment out if placing in a module):
# convert_fMRIvols_to_parcels(
#     data_path="/path/to/fmri/files",
#     atlas_path="/path/to/AAL3.nii.gz",
#     filename_substring="fMRI_in_MNI_space",
#     n_parcels=170
# )

In [10]:
list_nifti_files("/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/AD", filename_substring="MNI")

['/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/AD/20227_rsfMRI_NIFTI/rsfmri_unzip/1565751_20227_2_0/fMRI/rfMRI.ica/filtered_func_data_clean_MNI.nii.gz',
 '/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/AD/20227_rsfMRI_NIFTI/rsfmri_unzip/1714333_20227_2_0/fMRI/rfMRI.ica/filtered_func_data_clean_MNI.nii.gz']

In [12]:
convert_fMRIvols_to_parcels(
    data_path="/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/AD",
    atlas_path="/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/AAL3.nii.gz",
    filename_substring="MNI")

Found 2 files in '/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/AD' matching criteria.
Atlas loaded successfully from: /blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/AAL3.nii.gz
Processing file: filtered_func_data_clean_MNI.nii.gz


/scratch/local/57213319/ipykernel_3298062/1624577564.py:83: RuntimeWarning: Mean of empty slice
  pmTS[:, i - 1] = np.nanmean(y, axis=1)


Saved parcellated time series to: /blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/AD/20227_rsfMRI_NIFTI/rsfmri_unzip/1565751_20227_2_0/fMRI/rfMRI.ica/filtered_func_data_clean_MNI.dat
Processing file: filtered_func_data_clean_MNI.nii.gz
Saved parcellated time series to: /blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/AD/20227_rsfMRI_NIFTI/rsfmri_unzip/1714333_20227_2_0/fMRI/rfMRI.ica/filtered_func_data_clean_MNI.dat


In [13]:
def load_dat_file(filepath: str, delimiter: str = '\t') -> np.ndarray:
    """
    Load a .dat file into a NumPy array.

    Args:
        filepath (str): Path to the .dat file.
        delimiter (str, optional): The delimiter used in the .dat file. 
                                   Default is tab ('\\t').

    Returns:
        np.ndarray: The data from the .dat file as a NumPy array.
    """
    # loadtxt will automatically infer rows/columns based on the file
    data = np.loadtxt(filepath, delimiter=delimiter)
    return data

# Example usage:
# file_path = 'path/to/your_file.dat'
# data_array = load_dat_file(file_path)
# print("Data shape:", data_array.shape)
# print("First few rows:\n", data_array[:5])

In [14]:
data_array = load_dat_file("/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/UKB/brain/AD/20227_rsfMRI_NIFTI/rsfmri_unzip/1714333_20227_2_0/fMRI/rfMRI.ica/filtered_func_data_clean_MNI.dat")

In [20]:
data_array[0]

array([ 3449.31599414, 12473.73990467,  4350.95887865, 12421.64366449,
        1927.16029283, 16334.34015927,  5245.23351291, 12359.12260641,
        2621.93731137, 15005.94579193,  4085.93403897, 11940.01182825,
        9313.37808276,  9635.87994114,  8806.6658211 ,  8217.05946651,
        7510.79829706,  5344.61072954, 10649.92519008, 12240.03293291,
       13377.09387105, 12673.46547022,  8865.89300981,  7488.04881346,
        9687.19011353,  7511.6218072 , 10796.89003848, 10814.41069671,
        6132.69113849,  5937.14096227,   501.46699349, 11319.03961312,
       10204.11228984,  7453.49134059,     0.        ,     0.        ,
       10619.37272588, 10059.53513475,  8889.12476377,  7287.1945604 ,
        7484.7180828 ,  4427.06093033,  6461.45611909,  2854.96604595,
        7918.11534313,  2013.00464248, 10053.66012138,  7541.42851599,
       12176.33087074, 10684.87680499,  8022.25381789,  6724.4438351 ,
       15635.50849059, 11992.02555131, 11747.81896868, 11028.74523961,
      